In [ ]:
import uuid

from webullsdkcore.client import ApiClient
from webullsdktrade.api import API
from webullsdkcore.common.region import Region
from dotenv import load_dotenv
import os
load_dotenv()
your_app_key = os.getenv("WEBULL_APP_KEY")
your_app_secret = os.getenv("WEBULL_APP_SECRET")


In [ ]:
# this block give 401 error in UAT env

import uuid

from webullsdkcore.client import ApiClient
from webullsdktrade.api import API
from webullsdkcore.common.region import Region

optional_api_endpoint = "hk-openapi.uat.webullbroker.com"

api_client = ApiClient(your_app_key, your_app_secret, Region.HK.value)
api_client.add_endpoint(Region.HK.value, optional_api_endpoint)

class TestOrder(object):
    def __init__(self):
        self.api = API(api_client)

    def order(self):
        res = self.api.account.get_app_subscriptions()
        account_id = None
        if res.status_code == 200:
            print('app subscriptions:', res.json())
            result = res.json()
            account_id = result[0]['account_id']
            print("account id:", account_id)

        if account_id is None:
            print("account id is null")
            return

        client_order_id = uuid.uuid4().hex
        print('client order id:', client_order_id)
        stock_order = {
            "account_id": account_id,
            "stock_order": {
                "client_order_id": client_order_id,
                "instrument_id": "913256409",
                "side": "BUY",
                "tif": "DAY",
                "order_type": "AT_AUCTION",
                # "limit_price": "385.000",
                "qty": "1",
                "extended_hours_trading": False
            }
        }
        res = self.api.order.place_order(stock_order['account_id'], **stock_order['stock_order'])
        if res.status_code == 200:
            print('place order res:', res.json())


if __name__ == '__main__':
    openapi = TestOrder()
    openapi.order()




In [ ]:
from webullsdkcore.client import ApiClient
from webullsdktrade.api import API
from webullsdkcore.common.region import Region

your_app_key = os.getenv("WEBULL_APP_KEY")
your_app_secret = os.getenv("WEBULL_APP_SECRET")
api_client = ApiClient(your_app_key, your_app_secret, Region.HK.value)
api = API(api_client)
if __name__ == '__main__':
    res = api.account.get_app_subscriptions()
    account_id = None
    if res.status_code == 200:
        print('app subscriptions:', res.json())
        result = res.json()
        account_id = result[0]['account_id']
        print("account id:", account_id)

    if account_id is None:
        print("account id is null")

In [ ]:
import logging

from webullsdktradeeventscore.events_client import EventsClient
from webullsdkcore.common.region import Region

your_app_key = os.getenv("WEBULL_APP_KEY")
your_app_secret = os.getenv("WEBULL_APP_SECRET")
account_id = account_id


def _on_log(level, log_content):
    print(logging.getLevelName(level), log_content)


if __name__ == '__main__':
    client = EventsClient(your_app_key, your_app_secret,Region.HK.value)
    client.on_log = _on_log
    print("subscribe account", [account_id])
    client.do_subscribe([account_id])

In [ ]:
from webullsdkmdata.common.category import Category
from webullsdkmdata.common.subscribe_type import SubscribeType
from webullsdkmdata.quotes.subscribe.default_client import DefaultQuotesClient
from webullsdkcore.common.region import Region

your_app_key = os.getenv("WEBULL_APP_KEY")
your_app_secret = os.getenv("WEBULL_APP_SECRET")


def pt_logs(client, userdata, level, buf):
    print("userdata:%s, level:%s, buf:%s" % (userdata, level, buf))


def on_message(client, userdata, message):
    print("Received message '" + str(message.payload) + "' on topic '"
          + message.topic + "' with QoS " + str(message.qos))


if __name__ == '__main__':
    client = DefaultQuotesClient(your_app_key, your_app_secret, Region.HK.value)
    client.init_default_settings('00700', Category.HK_STOCK.name, SubscribeType.SNAPSHOT.name)
    client.on_log = pt_logs
    client.on_message = on_message
    client.connect_and_loop_forever()

In [ ]:
from webullsdktrade.api import API
from webullsdkcore.client import ApiClient
from webullsdkcore.common.region import Region
from webullsdkmdata.common.category import Category
symbol="00700"
symbols=[symbol]
category=Category.HK_STOCK.name
api_client = ApiClient(your_app_key, your_app_secret, Region.HK.value)
api = API(api_client)
response = api.instrument.get_instrument(symbols, category)
if response.status_code == 200:
    instruments = response.json()

In [ ]:
instruments

In [ ]:
from webullsdktrade.grpc_api import API
from webullsdkquotescore.grpc.grpc_client import GrpcApiClient
from webullsdkcore.common.region import Region

grpc_client = GrpcApiClient(your_app_key, your_app_secret, Region.HK.value)
api = API(grpc_client)
try: 
    response = api.market_data.get_quote(symbol, category)
    if response.status_code == 200:
        result = response.json()
except Exception as e:
    print(f"Error occurred: {e}")